In [2]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
import keras_lmu

2023-03-28 09:22:02.497267: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 09:22:03.031277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-28 09:22:03.031326: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-28 09:22:03.031331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [28]:
# Assuming the test set will have a similar storage format as the training set with a different file name and split name
def load_data(file_name, split_name):
    # Load data
    with open('Data/' + file_name, 'rb') as f:
        data = pickle.load(f)
    X = data[split_name]['images']
    y = data[split_name]['labels']
    return X, y

In [29]:
train_sequential_X, train_sequential_y = load_data('sequentialMNIST.pickle', 'train')
train_permuted_X, train_permuted_y = load_data('permutedMNIST.pickle', 'train')

In [7]:
# Stratified 80-20 split into train and validation sets
from sklearn.model_selection import train_test_split

train_sequential_X, val_sequential_X, train_sequential_y, val_sequential_y = train_test_split(train_sequential_X, train_sequential_y, test_size=0.2, stratify=train_sequential_y, random_state=42)
train_permuted_X, val_permuted_X, train_permuted_y, val_permuted_y = train_test_split(train_permuted_X, train_permuted_y, test_size=0.2, stratify=train_permuted_y, random_state=42)

# Check Shapes
print('Sequential MNIST')
print('Train X Shape: ', train_sequential_X.shape)
print('Train y Shape: ', train_sequential_y.shape)
print('Validation X Shape: ', val_sequential_X.shape)
print('Validation y Shape: ', val_sequential_y.shape)

print('Permuted MNIST')
print('Train X Shape: ', train_permuted_X.shape)
print('Train y Shape: ', train_permuted_y.shape)
print('Validation X Shape: ', val_permuted_X.shape)
print('Validation y Shape: ', val_permuted_y.shape)

Sequential MNIST
Train X Shape:  (48000, 784, 1)
Train y Shape:  (48000,)
Validation X Shape:  (12000, 784, 1)
Validation y Shape:  (12000,)
Permuted MNIST
Train X Shape:  (48000, 784, 1)
Train y Shape:  (48000,)
Validation X Shape:  (12000, 784, 1)
Validation y Shape:  (12000,)


# Loading the models

In [4]:
n_pixels = train_sequential_X.shape[1]

lmu_layer = tf.keras.layers.RNN(
    keras_lmu.LMUCell(
        memory_d=1,
        order=256,
        theta=n_pixels,
        hidden_cell=tf.keras.layers.SimpleRNNCell(212),
        hidden_to_memory=False,
        memory_to_memory=False,
        input_to_hidden=True,
        kernel_initializer="ones",
    )
)

# TensorFlow layer definition
inputs = tf.keras.Input((n_pixels, 1))
lmus = lmu_layer(inputs)
outputs = tf.keras.layers.Dense(10)(lmus)

# TensorFlow model definition
model_smnist = tf.keras.Model(inputs=inputs, outputs=outputs)
model_smnist.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)
print(model_smnist.summary())

saved_weights_fname_smnist = "Saves/sMNIST-weights.hdf5"
model_smnist.load_weights(saved_weights_fname_smnist)

2023-03-28 09:22:21.360624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 09:22:21.361671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 09:22:21.362864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 09:22:21.368791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-28 09:22:21.369836: I tensorflow/compiler/xla/stream_executo

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784, 1)]          0         
                                                                 
 rnn (RNN)                   (None, 212)               99641     
                                                                 
 dense (Dense)               (None, 10)                2130      
                                                                 
Total params: 101,771
Trainable params: 101,771
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
n_pixels = train_sequential_X.shape[1]

lmu_layer = tf.keras.layers.RNN(
    keras_lmu.LMUCell(
        memory_d=1,
        order=256,
        theta=n_pixels,
        hidden_cell=tf.keras.layers.SimpleRNNCell(212),
        hidden_to_memory=False,
        memory_to_memory=False,
        input_to_hidden=True,
        kernel_initializer="ones",
    )
)

# TensorFlow layer definition
inputs = tf.keras.Input((n_pixels, 1))
lmus = lmu_layer(inputs)
outputs = tf.keras.layers.Dense(10)(lmus)

# TensorFlow model definition
model_pmnist = tf.keras.Model(inputs=inputs, outputs=outputs)
model_pmnist.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"],
)
print(model_pmnist.summary())

saved_weights_fname_pmnist = "Saves/pMNIST-weights.hdf5"
model_pmnist.load_weights(saved_weights_fname_pmnist)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784, 1)]          0         
                                                                 
 rnn_1 (RNN)                 (None, 212)               99641     
                                                                 
 dense_1 (Dense)             (None, 10)                2130      
                                                                 
Total params: 101,771
Trainable params: 101,771
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
def perform_inference(model, data, labels):
    """
    Performs inference on the given data and labels.

    Assuming data to be a list of arrays of shape (n_pixels, 1).
    Assuming labels to be a list of integers.
    """
    predictions = model.predict(data)
    predictions = np.argmax(predictions, axis=1)
    report =  classification_report(labels, predictions, output_dict=True)
    return pd.DataFrame(report).transpose()

# Sample Predictions on Validation Set

In [23]:
sequential_val_report = perform_inference(model_smnist, val_sequential_X, val_sequential_y)

375/375 [==============================] - 54s 143ms/step


In [24]:
sequential_val_report

,precision,recall,f1-score,support
0,0.985762,0.993249,0.989491,1185.000000
1,0.987361,0.985163,0.986261,1348.000000
2,0.980392,0.964765,0.972516,1192.000000
3,0.966366,0.960848,0.963599,1226.000000
4,0.967660,0.973459,0.970551,1168.000000
5,0.971349,0.969557,0.970452,1084.000000
6,0.983885,0.979730,0.981803,1184.000000
7,0.974461,0.974461,0.974461,1253.000000
8,0.947368,0.969231,0.958175,1170.000000
9,0.964497,0.958824,0.961652,1190.000000


In [25]:
permuted_val_report = perform_inference(model_pmnist, val_permuted_X, val_permuted_y)

375/375 [==============================] - 54s 145ms/step


In [26]:
permuted_val_report

,precision,recall,f1-score,support
0,0.976150,0.967089,0.971598,1185.000000
1,0.974833,0.977003,0.975917,1348.000000
2,0.940486,0.941275,0.940881,1192.000000
3,0.917685,0.918434,0.918060,1226.000000
4,0.945407,0.934075,0.939707,1168.000000
5,0.927007,0.937269,0.932110,1084.000000
6,0.972485,0.955236,0.963784,1184.000000
7,0.958433,0.956903,0.957668,1253.000000
8,0.913880,0.934188,0.923922,1170.000000
9,0.923013,0.926891,0.924948,1190.000000
